In [1]:
import sys

sys.path.append('./drive/My Drive/Colab Notebooks/Modules')

In [1]:
import pandas as pd
import numpy as np
from preprocessing import get_vocab, index_sents
from embedding import create_embeddings
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
!pip install keras-tqdm
!pip install git+https://www.github.com/keras-team/keras-contrib.git
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers.wrappers import Bidirectional
from keras.layers import concatenate, Input, LSTM, Dropout, Embedding
from keras_contrib.layers import CRF
from keras_contrib.utils import save_load_utils
from gensim.models import Word2Vec
from keras_tqdm import TQDMNotebookCallback
from embedding import load_vocab

  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\camilo\appdata\local\temp\pip-req-build-vnmduso2
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101658 sha256=f8588f4e2437e05c8352ef29cd22ff7a787ceda18147265b65be66dd77d0fefb
  Stored in directory: C:\Users\Camilo\AppData\Local\Temp\pip-ephem-wheel-cache-navorxne\wheels\11\27\c8\4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git 'C:\Users\Camilo\AppData\Local\Temp\pip-req-build-vnmduso2'


In [3]:
data = pd.read_csv('./corpus/ancora/ancora_corpus_pos.csv')
del data['Unnamed: 0']

In [4]:
data.head()

,Sentence #,Word,Tag,POS
0,0,Las,O,DET
1,0,reservas,O,NOUN
2,0,de,O,ADP
3,0,oro,O,NOUN
4,0,y,O,CCONJ


In [5]:
sentmarks = data["Sentence #"].unique().tolist()
words = data["Word"].tolist()
postags = data["POS"].tolist()
nertags = data["Tag"].tolist()

In [6]:
%%time
sentence_text = []
sentence_post = []
sentence_ners = []

vocab = []

this_snt = []
this_pos = []
this_ner = []
#print(sentmarks[:10])
for idx, s in enumerate(sentmarks):
    #print(s)
    sentence_text.append(data[data['Sentence #'] == s]['Word'].tolist())
    sentence_post.append(data[data['Sentence #'] == s]['POS'].tolist())
    sentence_ners.append(data[data['Sentence #'] == s]['Tag'].tolist())
    #print(this_snt)

Wall time: 1min 12s


### Caracteres por sentencia

In [7]:
charSet = set()
for sentence in sentence_text:
  #print(sentence)
  for token in sentence:
      for char1 in token:
          charSet.add(char1)
chartList = list(charSet)


In [8]:
print(len(chartList))

113


In [9]:
char2Idx = {"PAD": 0, "UNK": 1}
for c in chartList:
    char2Idx[c] = len(char2Idx)

In [10]:
len(char2Idx)

115

In [11]:
max_len = 50

In [12]:
max_len_char = 50

Se calculan los caraceteres por token en cada setentencia

In [13]:
%%time
char_sentences = []
for sent in sentence_text:
  #print(sent)
  char_sent =[]
  for i in range(max_len):
    char_token = []
    for j in range(max_len_char):
      #print(j)
      try:
        char = sent[i][j]

        char_token.append(char2Idx.get(char))
      except:
        char_token.append(char2Idx.get('PAD'))
        #print('PAD')
    char_sent.append(char_token)
  char_sentences.append(np.array(char_sent))
      #if char in vocab_dict.keys()
     
  #for token in sent:
    #print(token)
  #  char_token = np.squeeze(index_sents(token, char2Idx))
    #print(char_token)
   # char_sent.append(char_token)
  #char_sent = np.squeeze(char_sent)
  #char_sentences.append(char_sent)


Wall time: 25.9 s


In [14]:
 idx2Char = {v: k for k, v in char2Idx.items()}

### Sentencias del conjunto de testeo
#### Obtener vocabulario y entradas de índice

necesitamos convertir la entrada de cadena a vectores enteros para la red de keras (la red pycrfsuite necesita cadenas, ya que extraerá vectores de características de las propias palabras).

indexaremos cada palabra desde 1 de acuerdo con la frecuencia inversa (la palabra más común es 1, etc.) hasta el tamaño máximo de vocabulario. Reservaremos dos espacios, 0 para el índice PAD y MAX_VOCAB-1 para palabras fuera de vocabulario o desconocidas (OOV / UNK). Como esto es algo aburrido, lo puse en funciones externas. Los paquetes como keras y sklearn tienen herramientas más robustas para esto, pero una palabra simple: el diccionario índice funcionará bien para este experimento

In [15]:
MAX_VOCAB = len(set(words))+2

In [16]:
word2idx, idx2word = get_vocab(sentence_text, MAX_VOCAB-2)
print(len(idx2word))

43005


In [17]:
# POS and NER tag vocab dicts
pos2idx, idx2pos = get_vocab(sentence_post, len(set(postags)))
ner2idx, idx2ner = get_vocab(sentence_ners, len(set(nertags))+2)
print(len(ner2idx))

11


In [18]:
# index
sentence_text_idx = index_sents(sentence_text, word2idx)
sentence_post_idx = index_sents(sentence_post, pos2idx)
sentence_ners_idx = index_sents(sentence_ners, ner2idx)
#print(sentence_post_idx)
print(len(sentence_post_idx))

17345


### División de los conjuntos de prueba y entrenamiento.
Dividimos los datos de entrenamiento en datos de entrenamiento y datos de prueba. los datos de prueba se usan solo para verificar el rendimiento del modelo. Un tercer conjunto, el conjunto de validación, puede separarse de nuestros datos de entrenamiento para el ajuste de hiperparámetros, aunque si utilizamos la validación cruzada k-fold, nuestro conjunto de validación cambiará cada vez.

In [19]:
TEST_SIZE = 0.15

In [20]:
indices = [i for i in range(len(sentence_text))]
#print(sentence_post_idx)

#print(train_idx)
#test_size=TEST_SIZE
train_idx, test_idx, X_train_pos, X_test_pos, X_train_char, X_test_char = train_test_split(indices,sentence_post_idx, char_sentences ,test_size=TEST_SIZE)
#X_train_pos,X_test_pos=train_test_split(indices, sentence_post_idx1 ,test_size=0.0001)


def get_sublist(lst, indices):
    result = []
    for idx in indices:
        result.append(lst[idx])
    return result

X_train_sents = get_sublist(sentence_text_idx, train_idx)
X_test_sents = get_sublist(sentence_text_idx, test_idx)
y_train_ner = get_sublist(sentence_ners_idx, train_idx)
y_test_ner = get_sublist(sentence_ners_idx, test_idx)

In [21]:
type(X_train_sents)

list

In [22]:
X_train_sents[0]

array([   34,    18,   382,  2330,    17,     3,  1414, 13183,  4564,
          29, 13641,    30,     2,     6, 13642,  8660,     5,    13,
        9797,    12,   581,  6650,     2,    10,  1577, 29350,     8,
       29351,    10,     2,     5,     6,  1777,   218,     1,  4304,
           1,  1085,  1727,     7,    20,  2431,  5041,    27,  6594,
           7,   571,     1,   223,    12,  7403, 29352,    12, 29353,
           2,  8601,     7,    13, 13643,  1717,     9,   466,     1,
        1778,     7,    28,  2083,     2,    82,    13,   621,  6823,
          44,     9,   662,     4])

In [23]:
type( np.asarray(X_train_char))

numpy.ndarray

In [24]:
np.asarray(X_train_char).shape

(14743, 50, 50)

In [25]:
#X_train_char = sequence.pad_sequences(X_train_char, maxlen=30, truncating='post', padding='post')
#X_test_char = sequence.pad_sequences(X_test_char, maxlen=30, truncating='post', padding='post')

In [26]:
#X_train_char_1[0]

### Creamos word2vec embeddings para palabras, pos-tags
Se ha demostrado que el uso de vectores de incrustación pre-entrenados para inicializar la capa de incrustación ayuda a la capacitación para diversas tareas de etiquetado de secuencias, como el etiquetado de POS (Huang, Xu & Yu 2015; Ma & Hovy 2016) y el Reconocimiento de entidades con nombre para inglés (Ma & Hovy 2016 ; Lee Changki 2017) y japonés (Misawa, Taniguchi, Miura y Ohkuma 2017).

Como estamos usando las etiquetas POS como entrada secundaria, también entrenaremos un espacio de incrustación para estas. utilizaremos solo los datos de entrenamiento para crear las incrustaciones. Estoy usando Gensim para esta tarea, y estoy usando una función auxiliar para ajustar el Word2Vec que guarda la incrustación y también el diccionario de vocabulario. Se vectorizan 6185 sentencias y solo una de testeo que no se usa.

In [27]:
train_sent_texts = [sentence_text[idx] for idx in train_idx]
        
w2v_vocab, w2v_model = create_embeddings(train_sent_texts,
                       embeddings_path='./embeddings/text_embeddings_acn.gensimmodel',
                       vocab_path='./embeddings/text_mapping_anc.json',
                       size=300,
                       workers=4,
                       iter=20)

In [28]:

# pos embeddings
train_post_texts = [sentence_post[idx] for idx in train_idx]

w2v_pvocab, w2v_pmodel = create_embeddings(train_post_texts,
                         embeddings_path='./embeddings/pos_embeddings_anc.gensimmodel',
                         vocab_path='./embeddings/pos_mapping_anc.json',
                         size=300,
                         workers=4,
                         iter=20)

### save everything to numpy binaries for loading
granted, pickle would probably be more suitable for a lot of these things. but over-reliance on numpy binaries is a bad habit i've picked up.

In [29]:
def numpy_save(saves, names):
    for idx, item in enumerate(saves):
        np.save('./encoded/ancora/{0}.npy'.format(names[idx]), item)
    return

saves = [
vocab,
sentence_text_idx,
sentence_post_idx,
sentence_ners_idx,
word2idx, idx2word,
pos2idx, idx2pos,
ner2idx, idx2ner,
train_idx,
test_idx,
X_train_sents,
X_test_sents,
X_train_pos,
X_test_pos,
y_train_ner,
y_test_ner,
sentence_text,
sentence_post,
sentence_ners,
idx2Char, char2Idx
]

names = [
'vocab',
'sentence_text_idx',
'sentence_post_idx',
'sentence_ners_idx',
'word2idx', 'idx2word',
'pos2idx', 'idx2pos',
'ner2idx', 'idx2ner',
'train_idx',
'test_idx',
'X_train_sents',
'X_test_sents',
'X_train_pos',
'X_test_pos',
'y_train_ner',
'y_test_ner',
'sentence_text',
'sentence_post',
'sentence_ners',
'idx2char', 'char2idx']

numpy_save(saves, names)

### LOAD NPY

In [ ]:
path_drive = './drive/My Drive/Colab Notebooks/' 

In [31]:

# network hyperparameters
MAX_LENGTH = 50
MAX_VOCAB = 43007    # see preprocessing.ipynb
WORDEMBED_SIZE = 300 # see data_preprocessing.ipynb
POS_EMBED_SIZE = 300 # see data_preprocessing.ipynb
HIDDEN_SIZE = 400    # LSTM Nodes/Features/Dimension
BATCH_SIZE = 64
DROPOUTRATE = 0.25
MAX_EPOCHS = 8       # max iterations, early stop condition below
#print(y_test_ner)

In [ ]:
# load data from npys (see preprocessing.ipynb)
print("loading data...\n")
vocab = list(np.load(path_drive+'encoded/vocab.npy',allow_pickle=True))
sentence_text = list(np.load(path_drive+'encoded/sentence_text.npy',allow_pickle=True))
sentence_post = list(np.load(path_drive+'encoded/sentence_post.npy',allow_pickle=True))
sentence_ners = list(np.load(path_drive+'encoded/sentence_ners.npy',allow_pickle=True))
sentence_text_idx = np.load(path_drive+'encoded/sentence_text_idx.npy',allow_pickle=True)
sentence_post_idx = np.load(path_drive+'encoded/sentence_post_idx.npy',allow_pickle=True)
sentence_ners_idx = np.load(path_drive+'encoded/sentence_ners_idx.npy',allow_pickle=True)
word2idx = np.load(path_drive+'encoded/word2idx.npy',allow_pickle=True).item()
idx2word = np.load(path_drive+'encoded/idx2word.npy',allow_pickle=True).item()
pos2idx = np.load(path_drive+'encoded/pos2idx.npy',allow_pickle=True).item()
idx2pos = np.load(path_drive+'encoded/idx2pos.npy',allow_pickle=True).item()
ner2idx = np.load(path_drive+'encoded/ner2idx.npy',allow_pickle=True).item()
idx2ner = np.load(path_drive+'encoded/idx2ner.npy',allow_pickle=True).item()
train_idx = np.load(path_drive+'encoded/train_idx.npy',allow_pickle=True)
test_idx = np.load(path_drive+'encoded/test_idx.npy',allow_pickle=True)
X_train_sents = np.load(path_drive+'encoded/X_train_sents.npy',allow_pickle=True)
X_test_sents = np.load(path_drive+'encoded/X_test_sents.npy',allow_pickle=True)
X_train_pos = np.load(path_drive+'encoded/X_train_pos.npy',allow_pickle=True)
X_test_pos = np.load(path_drive+'encoded/X_test_pos.npy',allow_pickle=True)
y_train_ner = np.load(path_drive+'encoded/y_train_ner.npy',allow_pickle=True)
y_test_ner = np.load(path_drive+'encoded/y_test_ner.npy',allow_pickle=True)
print(y_test_ner)

loading data...

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1])
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
 ...
 array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 4, 1, 1, 1, 1, 1, 1])
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 6, 1])
 array([4, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]


In [ ]:
from embedding import load_vocab
w2v_vocab, _ = load_vocab(path_drive+'Embeddings/text_mapping.json')
w2v_model = Word2Vec.load(path_drive+'Embeddings/text_embeddings.gensimmodel')
w2v_pvocab, _ = load_vocab(path_drive+'Embeddings/pos_mapping.json')
w2v_pmodel = Word2Vec.load(path_drive+'Embeddings/pos_embeddings.gensimmodel')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#X_train_char[0]

### Secuencias de pad
debemos 'rellenar' nuestras secuencias de entrada y salida a una longitud fija debido a la representación de gráfico fijo de Tensorflow.

In [32]:
# zero-pad the sequences to max length
print("zero-padding sequences...\n")
X_train_sents = sequence.pad_sequences(X_train_sents, maxlen=MAX_LENGTH, truncating='post', padding='post')
X_test_sents = sequence.pad_sequences(X_test_sents, maxlen=MAX_LENGTH, truncating='post', padding='post')
X_train_pos = sequence.pad_sequences(X_train_pos, maxlen=MAX_LENGTH, truncating='post', padding='post')
X_test_pos = sequence.pad_sequences(X_test_pos, maxlen=MAX_LENGTH, truncating='post', padding='post')
y_train_ner = sequence.pad_sequences(y_train_ner, maxlen=MAX_LENGTH, truncating='post', padding='post')
y_test_ner = sequence.pad_sequences(y_test_ner, maxlen=MAX_LENGTH, truncating='post', padding='post')

print(len(X_train_sents))
print(len(X_test_sents))
print(len(X_train_pos))
print(len(X_test_pos))
print(len(y_train_ner))
print(len(y_test_ner))

#print(y_train_ner)
print(y_test_ner)

zero-padding sequences...

14743
2602
14743
2602
14743
2602
[[1 4 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


In [ ]:
#X_train_char[0]

In [ ]:
#idx2Char[32]

In [33]:
y_ner=y_test_ner
print(y_ner)

[[1 4 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]]


In [34]:
# get the size of pos-tags, ner tags
TAG_VOCAB = len(list(idx2pos.keys()))
NER_VOCAB = len(list(idx2ner.keys()))
CHAR_VOCAB = len(list(idx2Char.keys()))

In [35]:
# reshape data for CRF
y_train_ner = y_train_ner[:, :, np.newaxis]
y_test_ner = y_test_ner[:, :, np.newaxis]

### Precargar las incrustaciones pre-entrenadas
Como se vio en estudios previos como Ma & Hovy 2016, se ha demostrado que cargar la capa de embeddings con vectores de embeddings preentrenados mejora el rendimiento de la red. Aquí inicializamos un embeddings en ceros y luego cargamos el embeddings desde el modelo previamente entrenado (si existe; puede que no se deba a los parámetros de Word2Vec).

In [36]:

# create embedding matrices from custom pretrained word2vec embeddings
word_embedding_matrix = np.zeros((MAX_VOCAB, WORDEMBED_SIZE))
c = 0
for word in word2idx.keys():
    # get the word vector from the embedding model
    # if it's there (check against vocab list)
    if word in w2v_vocab:
        c += 1
        # get the word vector
        word_vector = w2v_model[word]
        # slot it in at the proper index
        word_embedding_matrix[word2idx[word]] = word_vector
print("adicionados", c, "vectores")

adicionados8450vectores


In [37]:
pos_embedding_matrix = np.zeros((TAG_VOCAB, POS_EMBED_SIZE))
c = 0
for word in pos2idx.keys():
    # get the word vector from the embedding model
    # if it's there (check against vocab list)
    if word in w2v_pvocab:
        c += 1
        # get the word vector
        word_vector = w2v_pmodel[word]
        # slot it in at the proper index
        pos_embedding_matrix[pos2idx[word]] = word_vector
print("adicionamos", c, "vectores")

adicionamos14vectores


In [38]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2263927790390038746
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6661821563
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8899923082343030611
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:06:00.0, compute capability: 6.1"
]


In [39]:
from keras.initializers import RandomUniform
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, \
    Flatten, concatenate

In [40]:
#X_train_char[0]

In [41]:
CONV_SIZE = 3

# MODELO BILSTM_CRF

In [42]:
# define model

# text layers : dense embedding > dropout > bi-LSTM
#char_input = Input(shape=(MAX_LENGTH,),  name='char_input')
char_input = Input(shape=(None, max_len_char),  name='char_input')
char_embed = TimeDistributed(Embedding(input_dim=CHAR_VOCAB,output_dim=300, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5), name="Character_embedding"))(char_input)
char_drpot = Dropout(DROPOUTRATE, name='char_dropout', trainable=True)(char_embed)

conv1d_out = TimeDistributed(Conv1D(kernel_size=CONV_SIZE, filters=100, padding='same', activation='tanh', strides=1), name="Convolution")(char_drpot)
maxpool_out = TimeDistributed(MaxPooling1D(max_len_char), name="Maxpool")(conv1d_out)
char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
char = Dropout(DROPOUTRATE)(char)

txt_input = Input(shape=(None,), name='txt_input')
txt_embed = Embedding(MAX_VOCAB, WORDEMBED_SIZE, #input_length=MAX_LENGTH,
                      weights=[word_embedding_matrix],
                      name='txt_embedding', trainable=True)(txt_input)
txt_drpot = Dropout(DROPOUTRATE, name='txt_dropout')(txt_embed)

# pos layers : dense embedding > dropout > bi-LSTM
pos_input = Input(shape=(None,), name='pos_input')
pos_embed = Embedding(TAG_VOCAB, POS_EMBED_SIZE, #input_length=MAX_LENGTH,
                      weights=[pos_embedding_matrix],
                      name='pos_embedding', trainable=True)(pos_input)
pos_drpot = Dropout(DROPOUTRATE, name='pos_dropout')(pos_embed)

# merged layers : merge (concat, average...) word and pos > bi-LSTM > bi-LSTM
#mrg_cncat = concatenate([txt_drpot, pos_drpot, char_drpot], axis=2)
mrg_cncat = concatenate([txt_drpot, pos_drpot, char])
mrg_lstml = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True),
                          name='mrg_bidirectional_1')(mrg_cncat)

# extra LSTM layer, if wanted
mrg_drpot = Dropout(DROPOUTRATE, name='mrg_dropout')(mrg_lstml)
mrg_lstml = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True),
                          name='mrg_bidirectional_2')(mrg_lstml)


# final linear chain CRF layer
crf = CRF(NER_VOCAB, sparse_target=True)
mrg_chain = crf(mrg_lstml)

model = Model(inputs=[txt_input, pos_input, char_input], outputs=mrg_chain)

model.compile(optimizer='adam',
              loss=crf.loss_function,
              metrics=[crf.accuracy])

In [43]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to
char_input (InputLayer)         (None, None, 50)     0
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 50, 300 34500       char_input[0][0]
__________________________________________________________________________________________________
char_dropout (Dropout)          (None, None, 50, 300 0           time_distributed_1[0][0]
__________________________________________________________________________________________________
Convolution (TimeDistributed)   (None, None, 50, 100 90100       char_dropout[0][0]
__________________________________________________________________________________________________
txt_input (InputLayer)          (None, None)         0
____________________________________________

In [ ]:
print(len(X_train_sents),len(X_train_pos),len(y_train_ner))

14743 14743 14743


In [ ]:
X_train_sents.shape

(14743, 50)

In [ ]:
X_train_pos.shape

(14743, 50)

In [ ]:
type(X_train_pos)

numpy.ndarray

In [ ]:
type(X_train_char)

list

In [44]:
import json

# lets assume `model` is main model 
model_json = model.to_json()
with open("./models/model_in_json_ancora.json", "w") as json_file:
    json.dump(model_json, json_file)

In [ ]:
history = model.fit([X_train_sents, X_train_pos, np.asarray(X_train_char)], y_train_ner,
                    batch_size=BATCH_SIZE,
                    epochs=10,

                    verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
14743/14743 [==============================] - 123s 8ms/step - loss: 0.1246 - crf_viterbi_accuracy: 0.9625
Epoch 2/10
14743/14743 [==============================] - 115s 8ms/step - loss: 0.0374 - crf_viterbi_accuracy: 0.9886
Epoch 3/10
14743/14743 [==============================] - 112s 8ms/step - loss: 0.0214 - crf_viterbi_accuracy: 0.9932
Epoch 4/10
14743/14743 [==============================] - 114s 8ms/step - loss: 0.0102 - crf_viterbi_accuracy: 0.9958
Epoch 5/10
14743/14743 [==============================] - 113s 8ms/step - loss: 0.0022 - crf_viterbi_accuracy: 0.9974
Epoch 6/10
14743/14743 [==============================] - 114s 8ms/step - loss: -0.0037 - crf_viterbi_accuracy: 0.9983
Epoch 7/10
14743/14743 [==============================] - 112s 8ms/step - loss: -0.0090 - crf_viterbi_accuracy: 0.9989
Epoch 8/10
14743/14743 [==============================] - 114s 8ms/step - loss: -0.0137 - crf_viterbi_accuracy: 0.9992
Epoch 9/10
14743/14743 [=============================

In [ ]:
print(history.history.keys())

dict_keys(['loss', 'crf_viterbi_accuracy'])


In [ ]:
hist_dict = history.history

In [ ]:

# save the model
# because we are using keras-contrib, we must save weights like this, and load into network
# (see decoding.ipynb)
save_load_utils.save_all_weights(model, path_drive+'models/crf_model_char_ancora_10.h5')
np.save(path_drive + 'models/hist_dict_char_ancora_10.npy', hist_dict)
print("models saved!\n")

models saved!



In [ ]:
print(len(X_test_sents),len(X_test_pos))
print(X_test_sents)
print(X_test_pos)

2602 2602
[[ 1319   167    38 ...  1310    25    12]
 [42423    97     1 ...     0     0     0]
 [   23   648  1783 ...     0     0     0]
 ...
 [  410  3524    26 ...     0     0     0]
 [   10    85    71 ...     0     0     0]
 [10349     2  4169 ...   133     5     3]]
[[10 13  1 ...  1 10  2]
 [ 5  8  2 ...  0  0  0]
 [ 3  1  7 ...  0  0  0]
 ...
 [ 3  1 10 ...  0  0  0]
 [ 4 15  9 ...  0  0  0]
 [ 9  4  5 ... 11 12  3]]


In [ ]:
preds = model.predict([X_test_sents, X_test_pos, np.asarray(X_test_char)])

In [ ]:
preds = np.argmax(preds, axis=-1)
preds.shape

(2602, 50)

In [ ]:
trues = np.squeeze(y_test_ner, axis=-1)
trues.shape

(2602, 50)

In [ ]:
s_preds = [[idx2ner[t] for t in s] for s in preds]

s_trues = [[idx2ner[t] for t in s] for s in trues]

In [ ]:
!pip install seqeval
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(s_preds, s_trues)))
print("Recall-score: {:.1%}".format(recall_score(s_preds, s_trues)))
print("Precision-score: {:.1%}".format(precision_score(s_preds, s_trues)))

F1-score: 90.5%
Recall-score: 90.1%
Precision-score: 90.9%


In [ ]:
print(classification_report(s_trues, s_preds))

           precision    recall  f1-score   support

      PER       0.90      0.95      0.92      1405
      PAD       1.00      1.00      1.00      2223
      ORG       0.87      0.81      0.83      1539
     MISC       0.88      0.89      0.88      1633
      LOC       0.76      0.83      0.80       860

micro avg       0.90      0.91      0.90      7660
macro avg       0.90      0.91      0.90      7660



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
def bio_classification_report(y_true, y_pred):
    """
    from scrapinghub's python-crfsuite example
    
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O', 'PAD'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [ ]:
print(bio_classification_report(s_trues, s_preds))

              precision    recall  f1-score   support

       B-LOC       0.78      0.85      0.81       860
       I-LOC       0.72      0.75      0.74       285
      B-MISC       0.91      0.91      0.91      1633
      I-MISC       0.88      0.74      0.80       833
       B-ORG       0.91      0.83      0.87      1539
       I-ORG       0.86      0.86      0.86       973
       B-PER       0.91      0.96      0.93      1405
       I-PER       0.94      0.95      0.95       918

   micro avg       0.88      0.87      0.88      8446
   macro avg       0.86      0.86      0.86      8446
weighted avg       0.88      0.87      0.88      8446
 samples avg       0.06      0.06      0.06      8446



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#print(y_ner)
print(X_test_sents[:500])
print(len(X_test_sents[:500]))
decoded = []
for sent_idx in range(len(X_test_sents[:500])):
    
    this_txt = sequence.pad_sequences([X_test_sents[sent_idx]], maxlen=MAX_LENGTH, truncating='post', padding='post')
    this_pos = sequence.pad_sequences([X_test_pos[sent_idx]], maxlen=MAX_LENGTH, truncating='post', padding='post')
    this_pred = model.predict([this_txt, this_pos])
    this_pred = [np.argmax(p) for p in this_pred[0]]
    np.shape(this_pred)
    #print(this_pred)
    # for each word in the sentence...
    word, pos, tru, prd = [], [], [], []
    for idx, wordid in enumerate(X_test_sents[sent_idx][:len(this_pred)]):
        # decode word
        word.append(idx2word[wordid])
        # decode pos
        #print(X_test_pos[sent_idx][idx])
        pos.append(idx2pos[X_test_pos[sent_idx][idx]])
        # decode true NER tag
        #print(pos)
        #print(y_ner[sent_idx][idx])
        tru.append(idx2ner[y_ner[sent_idx][idx]])
        # decode prediction
        #print(tru)
        prd.append(idx2ner[this_pred[idx]])
        #print(prd)
    
    answ = pd.DataFrame(
    {
        'word': word,
        'pos': pos,
        'true': tru,
        'pred': prd,
        'skip' : [' ' for s in word]
    })
    answ = answ[['word', 'pos', 'true', 'pred', 'skip']]
    answ = answ.T
    decoded.append(answ)

[[ 1319   167    38 ...  1310    25    12]
 [42423    97     1 ...     0     0     0]
 [   23   648  1783 ...     0     0     0]
 ...
 [10505  1042     1 ...     3   302     9]
 [24137    17 16069 ...     0     0     0]
 [   10   822    69 ...     0     0     0]]
500


In [ ]:
result = pd.concat(decoded)

In [ ]:
print(result.shape)

(2500, 30)


In [ ]:
result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
word,Hace,cuatro,años,el,déficit,era,del,7,por,ciento,del,PIB,y,la,Seguridad,Social,estaba,"""",en,quiebra,"""",y,hoy,",",recordó,",",el,déficit,es,del
pos,AUX,NUM,NOUN,DET,NOUN,AUX,ADP,NUM,ADP,NUM,ADP,PROPN,CCONJ,DET,PROPN,PROPN,AUX,PUNCT,ADP,NOUN,PUNCT,CCONJ,ADV,PUNCT,VERB,PUNCT,DET,NOUN,AUX,ADP
true,O,O,O,O,O,O,O,B-MISC,I-MISC,I-MISC,O,B-MISC,O,O,B-ORG,I-ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O
pred,O,O,O,O,O,O,O,B-MISC,I-MISC,I-MISC,O,B-MISC,O,O,B-ORG,I-ORG,O,O,O,O,O,O,O,O,O,O,O,O,O,O
skip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## Etiquetado de texto

In [ ]:
from nltk.tag import StanfordPOSTagger
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
!wget 'https://nlp.stanford.edu/software/stanford-tagger-4.0.0.zip'
!unzip stanford-tagger-4.0.0.zip
nltk.download('punkt')

--2020-07-13 22:34:50--  https://nlp.stanford.edu/software/stanford-tagger-4.0.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78260695 (75M) [application/zip]
Saving to: ‘stanford-tagger-4.0.0.zip’

stanford-tagger-4.0 100%[===================>]  74.63M  57.5MB/s    in 1.3s    

2020-07-13 22:34:51 (57.5 MB/s) - ‘stanford-tagger-4.0.0.zip’ saved [78260695/78260695]

Archive:  stanford-tagger-4.0.0.zip
   creating: stanford-tagger-4.0.0/
  inflating: stanford-tagger-4.0.0/stanford-postagger-4.0.0-javadoc.jar  
  inflating: stanford-tagger-4.0.0/stanford-postagger.bat  
  inflating: stanford-tagger-4.0.0/sample-input.txt  
  inflating: stanford-tagger-4.0.0/sample-output.txt  
   creating: stanford-tagger-4.0.0/data/
  inflating: stanford-tagger-4.0.0/data/enclitic-inflections.data  
  inflating: stanford-tagger-4.0.0/stanford-pos

True

In [ ]:
st = StanfordPOSTagger('/content/stanford-tagger-4.0.0/models/spanish-ud.tagger','/content/stanford-tagger-4.0.0/stanford-postagger.jar')

/usr/local/lib/python3.6/dist-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [ ]:
text = '''Para ser dos países con una historia, circunstancias y geografía tan parecidas, Canadá y Estados Unidos han terminado teniendo una identidad nacional bien distinta.

Los dos gigantes norteamericanos comparten un continente, 8.893 kilómetros de frontera terrestre -la más larga del mundo- y una historia llena de paralelos y similitudes.

Ambos empezaron como naciones fundadas por europeos en terrenos arrebatados a los indígenas pero eventualmente, gracias a la inmigración global, se convirtieron en el último siglo en sociedades enormemente multirraciales.'''

In [ ]:
from nltk import sent_tokenize

In [ ]:
sample_sentences = sent_tokenize(text)

In [ ]:
postag = [st.tag(s.split()) for s in sample_sentences ]

In [ ]:
word_sample = []
pos_sample = []
for input in postag:
  tmp_words = []
  tmp_pos = []
  for word, pos in input:
    tmp_words.append(word)
    tmp_pos.append(pos)
  word_sample.append(tmp_words)
  pos_sample.append(tmp_pos)

In [ ]:
def GetCharSentence(sentences):
  char_sentences = []
  for sent in sentences:
    char_sent = []
    for i in range(MAX_LENGTH):
      char_token = []
      for j in range(max_len_char):
        #print(j)
        try:
          char = sent[i][j]

          char_token.append(char2Idx.get(char))
        except:
          char_token.append(char2Idx.get('PAD'))
          #print('PAD')
      char_sent.append(char_token)
    char_sentences.append(np.array(char_sent))
  return char_sentences

In [ ]:
char_test = GetCharSentence(word_sample)

In [ ]:
char_test[0][0]

array([ 66,  67, 107,  67,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
idx2Char[107]

'r'

In [ ]:
print(word_sample)
print(pos_sample)

[['Para', 'ser', 'dos', 'países', 'con', 'una', 'historia,', 'circunstancias', 'y', 'geografía', 'tan', 'parecidas,', 'Canadá', 'y', 'Estados', 'Unidos', 'han', 'terminado', 'teniendo', 'una', 'identidad', 'nacional', 'bien', 'distinta.'], ['Los', 'dos', 'gigantes', 'norteamericanos', 'comparten', 'un', 'continente,', '8.893', 'kilómetros', 'de', 'frontera', 'terrestre', '-la', 'más', 'larga', 'del', 'mundo-', 'y', 'una', 'historia', 'llena', 'de', 'paralelos', 'y', 'similitudes.'], ['Ambos', 'empezaron', 'como', 'naciones', 'fundadas', 'por', 'europeos', 'en', 'terrenos', 'arrebatados', 'a', 'los', 'indígenas', 'pero', 'eventualmente,', 'gracias', 'a', 'la', 'inmigración', 'global,', 'se', 'convirtieron', 'en', 'el', 'último', 'siglo', 'en', 'sociedades', 'enormemente', 'multirraciales.']]
[['ADP', 'AUX', 'NUM', 'NOUN', 'ADP', 'DET', 'NUM', 'NOUN', 'CCONJ', 'NOUN', 'ADV', 'PROPN', 'PROPN', 'CCONJ', 'PROPN', 'PROPN', 'AUX', 'VERB', 'AUX', 'DET', 'NOUN', 'ADJ', 'ADV', 'PROPN'], ['DET', 

In [ ]:
sentences_sample_idx = index_sents(word_sample, word2idx)
post_sampple_idx = index_sents(pos_sample, pos2idx)

In [ ]:
X_sample_sents = sequence.pad_sequences(sentences_sample_idx, maxlen=MAX_LENGTH, truncating='post', padding='post')

In [ ]:
X_sample_sents[1]

array([   42,    36,  8704,  4366,  5923,    13, 43004, 43004,   584,
           1,  1258, 10075, 43004,    31,  1910,    12, 43004,     8,
          18,   336,  2563,     1,  8845,     8, 43004,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [ ]:
post_sampple_idx = sequence.pad_sequences(post_sampple_idx, maxlen=MAX_LENGTH, truncating='post', padding='post')

In [ ]:
predictions = model.predict([X_sample_sents,post_sampple_idx, np.asarray(char_test)])
#print(predictions, predictions.shape)

In [ ]:
#predictions

In [ ]:
predictions = np.argmax(predictions, axis=-1)

In [ ]:
predictions

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 9, 1, 7, 9, 1, 1, 1, 1, 1, 1,
        1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 7, 1, 1, 1, 1, 1,
        1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [ ]:
s_predictions = [[idx2ner[t] for t in s] for s in predictions]

In [ ]:
#!pip install tabulate
from tabulate import tabulate
for idx,s in enumerate(sample_sentences):
  heads1 = s.split()
  body1 = [s_predictions[idx][:len(s.split())]]
  print(tabulate(body1, headers=heads1))
  print('\n')

Para    ser    dos    países    con    una    historia,    circunstancias    y    geografía    tan    parecidas,    Canadá    y    Estados    Unidos    han    terminado    teniendo    una    identidad    nacional    bien    distinta.
------  -----  -----  --------  -----  -----  -----------  ----------------  ---  -----------  -----  ------------  --------  ---  ---------  --------  -----  -----------  ----------  -----  -----------  ----------  ------  -----------
O       O      O      O         O      O      O            O                 O    O            O      B-LOC         I-LOC     O    B-LOC      I-LOC     O      O            O           O      O            O           O       B-PER


Los    dos    gigantes    norteamericanos    comparten    un    continente,    8.893    kilómetros    de    frontera    terrestre    -la     más    larga    del    mundo-    y    una    historia    llena    de    paralelos    y    similitudes.
-----  -----  ----------  -----------------  ---------